In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython
import os
import time
import re
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from pearlsim.ml_utilities import *
import pickle

# Load Data
The data used in this notebook is created in the generate_kernal_burnup_data notebook. It requires running a lot of Serpent calculations, hence why I made it a separate notebook. First we load and examine the data.

The input features to this model include:
- Depletion Time: the time in days that the pebble has been exposed to flux
- Surface current: Labeled by energy bin, the number of incoming neutrons at a certain energy group imposing on the surface of the pebble per second
- Power: The power produced by the pebble in Watts
- Starting isotope concentrations: The amount of each nuclide present in the pebble in atom density. 

The output features to this model include:
- Ending isotope concentrations: The amount of each nuclide present in the pebble AFTER a burnup step.

Note that the nuclide concentrations and surface current values can be quite large and span orders of magnitude.

In [2]:
features = pd.read_csv("training_data/burnup_features.csv", index_col=0)
targets = pd.read_csv("training_data/burnup_target.csv", index_col=0)

In [3]:
features.iloc[0:10]

,depletion_time,2.9005e-08,9.9e-08,2.1e-07,4.525e-07,7.985000000000001e-07,1.061e-06,1.5025e-06,2.9275e-06,6.9385e-06,...,62147<lib>,62148<lib>,62149<lib>,62150<lib>,62151<lib>,621520,631510,92235<lib>,92238<lib>,power
0,4.600,2.126150e+14,5.195180e+14,5.748160e+14,3.973870e+14,1.264600e+14,2.376000e+13,8.956950e+13,1.416580e+14,1.624290e+14,...,2.135803e-08,3.553372e-08,8.929475e-08,0.000001,2.015638e-07,2.152646e-07,7.349988e-11,0.004016,0.018843,1896
1,3.000,2.301940e+14,5.653970e+14,6.157750e+14,4.178810e+14,1.131750e+14,2.829020e+13,1.121280e+14,1.649230e+14,1.915860e+14,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.004670,0.018973,1760
2,4.600,1.361290e+14,3.365870e+14,3.607000e+14,1.822650e+14,2.672060e+13,1.511560e+12,2.970530e+13,2.870360e+13,5.100380e+13,...,2.226552e-07,5.757140e-07,8.451241e-08,0.000004,2.399743e-07,9.411487e-07,2.997949e-10,0.003077,0.018654,2002
3,6.525,4.433100e+14,9.815350e+14,9.476020e+14,5.650410e+14,1.337090e+14,5.945700e+13,1.086090e+14,1.727270e+14,2.245720e+14,...,4.947674e-08,1.259036e-07,7.752272e-08,0.000003,2.264037e-07,5.364294e-07,9.774726e-11,0.003546,0.018794,1052
4,6.525,2.093670e+14,4.521590e+14,4.341860e+14,2.476010e+14,5.047690e+13,2.374190e+13,5.077140e+13,9.692880e+13,8.705620e+13,...,7.428893e-07,2.328590e-06,9.011014e-08,0.000008,2.453769e-07,1.935729e-06,6.223075e-10,0.002208,0.018457,1362
5,0.200,2.971010e+14,6.555050e+14,6.673100e+14,3.406630e+14,4.286510e+13,8.646920e+12,4.668620e+13,7.135180e+13,7.175540e+13,...,9.186741e-07,3.036580e-06,9.194794e-08,0.000009,2.463628e-07,2.384741e-06,6.883000e-10,0.001904,0.018371,1553
6,6.525,2.936950e+14,7.813940e+14,8.917750e+14,5.150800e+14,1.664900e+14,5.342390e+13,1.540780e+14,2.073200e+14,2.272590e+14,...,7.769062e-07,2.086122e-06,8.009705e-08,0.000007,2.430980e-07,1.784846e-06,6.126424e-10,0.002343,0.018508,2989
7,6.525,2.164250e+14,4.783660e+14,6.283170e+14,4.128660e+14,1.195120e+14,2.698790e+13,8.526910e+13,1.395310e+14,1.767970e+14,...,7.912550e-07,2.363068e-06,9.769413e-08,0.000008,2.466771e-07,1.971257e-06,6.331971e-10,0.002185,0.018455,2137
8,6.525,4.275320e+14,1.207050e+15,1.235520e+15,7.223650e+14,1.658150e+14,6.069740e+13,1.704880e+14,2.234050e+14,2.462940e+14,...,9.082656e-07,2.974320e-06,8.184917e-08,0.000009,2.482320e-07,2.467873e-06,7.002601e-10,0.001854,0.018356,1231
9,6.525,2.378630e+14,6.538530e+14,6.414590e+14,4.245040e+14,1.122480e+14,3.607920e+13,1.347680e+14,1.722720e+14,1.893370e+14,...,5.987565e-07,2.172235e-06,8.440447e-08,0.000008,2.528084e-07,1.954940e-06,4.990086e-10,0.002222,0.018485,2211


In [4]:
targets.iloc[0:10]

,10010,10020,10030,20030,20040,40090,50110,50120,6012<lib>,60130,...,621460,62147<lib>,62148<lib>,62149<lib>,62150<lib>,62151<lib>,621520,631510,92235<lib>,92238<lib>
0,6.769255e-11,3.563740e-13,2.012836e-15,7.172889e-19,6.572792e-06,1.104684e-06,0.0,0.000000e+00,0.011690,0.000131,...,9.818149e-12,3.272728e-08,5.299093e-08,3.045631e-07,1.151985e-06,2.522159e-07,2.157895e-07,9.579983e-11,0.003985,0.018783
1,6.265255e-13,5.075438e-15,5.106375e-17,7.772492e-21,4.492448e-06,7.602572e-07,0.0,0.000000e+00,0.011690,0.000129,...,1.066661e-15,1.498533e-11,4.693565e-12,2.435013e-08,5.899767e-11,1.229243e-08,6.458702e-11,3.052743e-13,0.004651,0.018936
2,3.201253e-10,2.142781e-12,4.027880e-15,1.995947e-18,8.794220e-06,1.482491e-06,0.0,0.000000e+00,0.011690,0.000133,...,1.156139e-10,2.579135e-07,6.506087e-07,3.573398e-07,3.861202e-06,2.933327e-07,9.415500e-07,3.259746e-10,0.003052,0.018581
3,1.098132e-10,5.730922e-13,2.965919e-15,1.533621e-18,6.058036e-06,1.017553e-06,0.0,0.000000e+00,0.011690,0.000130,...,2.201921e-11,8.205858e-08,1.770346e-07,4.442153e-07,2.539061e-06,2.942143e-07,5.367994e-07,1.349720e-10,0.003525,0.018743
4,6.473544e-10,5.617058e-11,5.426978e-15,3.628881e-17,9.110679e-06,1.526630e-06,0.0,1.028902e-240,0.011690,0.000133,...,3.855325e-10,8.198505e-07,2.496571e-06,3.790535e-07,7.641240e-06,2.922954e-07,1.936297e-06,6.596372e-10,0.002188,0.018380
5,2.050532e-10,7.132536e-11,8.495759e-15,1.182674e-16,4.444962e-07,6.089275e-08,0.0,7.296525e-199,0.011691,0.000126,...,1.729434e-11,9.210681e-07,3.049576e-06,1.202929e-07,9.176598e-06,2.496170e-07,2.384757e-06,6.893464e-10,0.001903,0.018368
6,1.634065e-09,1.208342e-11,7.386260e-15,1.408029e-17,2.017378e-05,3.395273e-06,0.0,0.000000e+00,0.011688,0.000142,...,8.979034e-10,8.551341e-07,2.183547e-06,3.474216e-07,7.201413e-06,3.178252e-07,1.786114e-06,6.509571e-10,0.002297,0.018341
7,1.251478e-09,4.898536e-11,8.102288e-15,3.850885e-17,1.522162e-05,2.551214e-06,0.0,0.000000e+00,0.011689,0.000138,...,6.849625e-10,8.678753e-07,2.591637e-06,5.308206e-07,7.720657e-06,3.169369e-07,1.972139e-06,6.723985e-10,0.002154,0.018333
8,8.057072e-10,1.063038e-11,4.186106e-15,1.921046e-17,9.005983e-06,1.508669e-06,0.0,0.000000e+00,0.011690,0.000133,...,4.625929e-10,9.924497e-07,3.169487e-06,4.024991e-07,9.157709e-06,2.940064e-07,2.468391e-06,7.379829e-10,0.001838,0.018283
9,7.822523e-10,6.315778e-12,7.560371e-15,4.899945e-18,1.521735e-05,2.554512e-06,0.0,0.000000e+00,0.011689,0.000138,...,5.286155e-10,6.770015e-07,2.339200e-06,4.478601e-07,7.965891e-06,3.224564e-07,1.955839e-06,5.388983e-10,0.002190,0.018360


# Data Standardization
Simple standardization is performed here along each column. Log-standardization on the nuclide concentrations and current values gave a massive improvement to the model accuracy for me. It may not be necessary for the current values.

In [5]:
train_split = 0.8
np.random.seed(42)

def standardize(raw_data, mean=None, std=None, axis=0):
    if mean is None:
        mean = np.mean(raw_data, axis = axis)
    if std is None:
        std = np.std(raw_data, axis = axis)
        std[ std==0 ] = 0.1
    result = (raw_data - mean) / std
    return result, mean, std

def unstandardize(standardized_data, mean, std):
    raw_data = (standardized_data*std)+mean
    return raw_data

log_features = features.apply(lambda x: np.log10(x + 1))
log_targets = targets
log_targets = targets.apply(lambda x: np.log10(x + 1))
log_features['power'] = features['power']
log_features['depletion_time'] = features['depletion_time']

num_data = len(features)
training_size = int(num_data*train_split)
testing_size = num_data - training_size
data_indices = np.arange(num_data)
training_indices = np.random.choice(num_data, training_size, replace=False)
testing_indices = data_indices[np.in1d(data_indices, training_indices, invert=True)]

training_data, data_mean, data_std = standardize(log_features.iloc[training_indices])
training_target, target_mean, target_std = standardize(log_targets.iloc[training_indices])
testing_data, _, _  = standardize(log_features.iloc[testing_indices], mean=data_mean, std=data_std)
testing_target, _, _  = standardize(log_targets.iloc[testing_indices], mean=target_mean, std=target_std)

print(np.shape(training_data))
print(np.shape(training_target))
print(np.shape(testing_data))
print(np.shape(testing_target))

(400, 424)
(400, 404)
(100, 424)
(100, 404)


# Model Training
I threw together a quick RFR model and got some results. You're free to change to any other type of model, as long as its something I can save and load into other modules. Things to try:
- Properly using cross validation
- Tuning the hyper parameters
- Trying a different model, probably a neural net

In [6]:
best_params = {'max_depth': 10, 
               'n_estimators': 1000, 
               'n_jobs': 20,} 
rfr_model = RandomForestRegressor(random_state=0)
rfr_model.set_params(**best_params)
rfr_model.fit(training_data, training_target)
rfr_model_test_score = rfr_model.score(testing_data, testing_target)
print(f"RFR score: {rfr_model_test_score}")

RFR score: 0.9097377320041374


/global/software/sl-7.x86_64/modules/langs/python/3.7/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


Next we save the models and standardization parameters so the model can be used in the simulation.

In [7]:
model_data = pickle.dumps(rfr_model)
with open("ml_models/burnup_rfr.pkl", 'wb') as f:
    f.write(model_data)
data_mean.to_csv("ml_models/burnup_rfr_data_mean.csv", header=True)
data_std.to_csv("ml_models/burnup_rfr_data_std.csv", header=True)
target_mean.to_csv("ml_models/burnup_rfr_target_mean.csv", header=True)
target_std.to_csv("ml_models/burnup_rfr_target_std.csv", header=True)

KeyboardInterrupt: 